# Exploration of the pictures dataset

Dans ce premier notebook sur l'exploration du dataset d'images, on va étudier les principales caractéristiques de ce dernier : valeurs manquantes, qualité des images, taille, etc.
